In [1]:
import tweepy
import csv
import sqlite3
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [2]:
#twitter
token_data = open("Tugas Sanbercode/twittertoken.csv")
tokens = csv.reader(token_data,delimiter=',')

data_token = [row[2] for row in tokens]
consumer_key = data_token[0]
consumer_secret = data_token[1]
access_token = data_token[2]
access_token_secret = data_token[3]

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [23]:
#sql connection
connect = sqlite3.connect("DB Browser SQLite/database_twitter.db")
cursor = connect.cursor()

In [4]:
class tugas:
    def __init__(self,topic='biden trump',max_tweet=200,start_date='2020-11-18',last_date='2020-11-27'):
        new_topic = topic+'-filter:retweets'
        #hasil = api.search(q=new_topic,count=max_tweet,tweet_mode='extended',lang='id',since=start_date,until=last_date)
        hasil = [status for status in tweepy.Cursor(api.search,q=new_topic,count=max_tweet,tweet_mode='extended',lang='id',since=start_date,until=last_date).items(max_tweet)]
        self.topic = topic
        self.max_tweet = max_tweet
        self.start_date = start_date
        self.last_date = last_date
        self.hasil = hasil
    
    def get_data(self):
        text = [tweet.full_text for tweet in self.hasil]
        tweetid = [tweet.id for tweet in self.hasil]
        userid = [tweet.user.id for tweet in self.hasil]
        created = [tweet.created_at for tweet in self.hasil]
        name = [tweet.user.name for tweet in self.hasil]
        screenname = [tweet.user.screen_name for tweet in self.hasil]
        location = [tweet.user.location for tweet in self.hasil]
        acc_created = [tweet.user.created_at for tweet in self.hasil]
        follower = [tweet.user.followers_count for tweet in self.hasil]
        friend = [tweet.user.friends_count for tweet in self.hasil]
        verified = [tweet.user.verified for tweet in self.hasil]
        return text,tweetid,userid,created,name,screenname,location,acc_created,follower,friend,verified
    
    def clean_data(self,data):
        #casefolding
        data1 = [item.lower() for item in data]
        data2 = [re.sub(r"\d+", "", item) for item in data1]
        data3 = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", item).split()) for item in data2]
        
        #stopword (nltk)
        output = []
        for string in data3:
            stop_words = set(stopwords.words('indonesian'))
            word_tokens = word_tokenize(string) 
            data4 = ' '.join([word for word in word_tokens if not word in stop_words])

        #stemming (sastrawi)
            factory = StemmerFactory()
            stemmer = factory.create_stemmer()
            data5 = stemmer.stem(data4)
            output.append(data5)
        return output
        
    def save_sql(self,clean_tweet,mode):
        cursor = connect.cursor()
        if mode=='lastscraping':
            scrap = 1
            last_scraping = self.start_date
            query_lastscraping = '''insert into Lastscraping (last_get,status) values (?,?);'''  
            value = (last_scraping, scrap)  
            cursor.execute(query_lastscraping,value)
                
        elif mode=='tweet':
            #get last scraping_id
            query_get_lastscraping = '''select max(scraping_id) from Lastscraping'''   
            cursor.execute(query_get_lastscraping) 
            scraping = cursor.fetchone()
            scraping = list(scraping)
            scraping = scraping[0]
            #get All tweet_id
            query_get_tweetid = '''select tweet_id from Tweet'''   
            cursor.execute(query_get_tweetid) 
            key1 = cursor.fetchall()
            key2 = [str(item) for item in key1]
            key3 = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", item).split()) for item in key2]
            key4 = [int(item) for item in key3]
            unique_tweet = key4
            data_ke = 0
            for tweet in self.hasil:
                if tweet.id not in unique_tweet:
                    unique_tweet.append(tweet.id)
                    query_tweet = '''insert into Tweet (tweet_id,user_id,date_created,tweet,cleantweet,scraping_id) values (?,?,?,?,?,?);'''  
                    value = (tweet.id,tweet.user.id,tweet.created_at,tweet.full_text,clean_tweet[data_ke],scraping)  
                    cursor.execute(query_tweet,value) 
                    data_ke +=1
            #return unique_tweet
        
        elif mode=='user':
            #get All user_id
            query_get_userid = '''select user_id from User'''   
            cursor.execute(query_get_userid) 
            key1 = cursor.fetchall()
            key2 = [str(item) for item in key1]
            key3 = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", item).split()) for item in key2]
            key4 = [int(item) for item in key3]
            unique_user = key4
            for tweet in self.hasil:
                if tweet.user.id not in unique_user:
                    unique_user.append(tweet.user.id)
                    query_user = '''insert into User values (?,?,?,?,?,?,?,?);'''  
                    value = (tweet.user.id,tweet.user.name,tweet.user.screen_name,tweet.user.location,tweet.user.created_at,tweet.user.followers_count,tweet.user.friends_count,tweet.user.verified)  
                    cursor.execute(query_user,value)
            #return unique_user
                
        else:
            pass
        connect.commit()
        cursor.close()
        #connect.close()
        
    def reset_scraping(self):
        cursor = connect.cursor()
        query_reset_lastscraping = '''UPDATE Lastscraping SET status=0;'''
        cursor.execute(query_reset_lastscraping)
        connect.commit()
        cursor.close()

In [5]:
data = tugas(start_date='2020-11-20',last_date='2020-11-21')
text,tweetid,userid,created,name,screenname,location,acc_created,follower,friend,verified = data.get_data()
print(text)
print(len(text))

['Biden: Penolakan Trump terhadap Pemilu Sangat Tidak Bertanggungjawab\n\nhttps://t.co/qjWAN66ePs', '@zittokabwe KWA HIYO NA MAREKANI HAWAPEWI MISAADA HIYO?MHE DONALD TRUMP ANALALAMIKA KAIBIWA KURA!ANADAI KWAMBA MHE JOE BIDEN ALIKUWA NA MABEGI YA KURA ZILIZOKWISHA PIGWA ZIKAWA ZINATUMBUKIZWA KWENYE MASANDUKU! KAMA DUNIA INGEFATA AKILI ZENU,HAKUNA NCHI HATA MOJA INGEPATA MISAADA YENU HIYO!!! https://t.co/Sz61QgIoPM', 'Ga cuma Trump,  Mr Joe Biden.. https://t.co/1aP3Rm6CtW', 'HASIL HITUNG ULANG SUARA DI GEORGIA, TRUMP TETAP KALAH\nBiden menyebut Trump sebagai presiden paling tidak bertanggung jawab dalam sejarah Amerika.\nhttps://t.co/MDAckLQF2d', "Joe Biden Sebut Donald Trump sebagai 'Presiden Paling Tidak Bertanggung Jawab dalam Sejarah AS'\nhttps://t.co/BcwQ9kKkeo", 'Geram, Biden Sebut Trump Keterlaluan https://t.co/1BxD2p60hb #Internasional', '@LynnRoseberry trump eai hiau kuan bi kok , joe biden xi chi kor kai nang, bei hiau kuan bi kok.', 'Biden Sebut Strategi Trump Menolak Mengaku

In [6]:
print(tweetid)

[1329693870033027074, 1329686294751039488, 1329684823351259136, 1329683970368303107, 1329680732474654722, 1329680132827607045, 1329678051131629569, 1329672328326705152, 1329671450979041280, 1329667406541639680, 1329664941838786565, 1329660967333060609, 1329659068097601538, 1329658577712074758, 1329656029496430593, 1329653379153289216, 1329652588078268417, 1329651833518706695, 1329651699011653632, 1329649656746418176, 1329649638085849088, 1329649618767011840, 1329646913604509698, 1329644778275827712, 1329643249812729857, 1329641342692016128, 1329640612568690690, 1329637976377200640, 1329634122944286723, 1329634122491301888, 1329632534141997056, 1329632316818337793, 1329631554142752768, 1329630379204984835, 1329629047664439297, 1329627510724722688, 1329626771575037953, 1329626394184224768, 1329625592514408449, 1329624758204428289, 1329622502847307776, 1329621494758998017, 1329620483189313536, 1329620456207388673, 1329619642969509889, 1329618603633647617, 1329618368442273792, 132961669768

In [7]:
print(userid)

[207854679, 911665393663660033, 1297510664148103168, 124171593, 58880688, 939083912575197186, 2455899126, 75077164, 58880688, 43639669, 1300659818529730560, 1176129395414765568, 731746252795174913, 731746252795174913, 1176129395414765568, 423324860, 46693036, 2614834184, 2430442939, 578059643, 578059643, 1090116885398646784, 58886762, 543920571, 1132166502247886850, 1328040153940516864, 786549338818224128, 829366551446818816, 598959076, 17128975, 18186609, 69183155, 2430442939, 497665754, 118646322, 141515518, 237493762, 350668803, 332563924, 23343960, 998688530, 118646322, 916530767156568069, 497665754, 336355440, 759692754985242625, 69183155, 522078771, 350668803, 2553884263, 909359407, 79130206, 558466886, 231355136, 423220305, 23343960, 3319260420, 2575194188, 22126902, 1048117926794948608, 102098902, 18071520, 368662151, 598959076, 1129936609, 23343960, 1324456125186613255, 1310723840956301312, 1315788739298369537, 30617522]


In [8]:
print(created)

[datetime.datetime(2020, 11, 20, 7, 51, 37), datetime.datetime(2020, 11, 20, 7, 21, 31), datetime.datetime(2020, 11, 20, 7, 15, 40), datetime.datetime(2020, 11, 20, 7, 12, 17), datetime.datetime(2020, 11, 20, 6, 59, 25), datetime.datetime(2020, 11, 20, 6, 57, 2), datetime.datetime(2020, 11, 20, 6, 48, 46), datetime.datetime(2020, 11, 20, 6, 26, 1), datetime.datetime(2020, 11, 20, 6, 22, 32), datetime.datetime(2020, 11, 20, 6, 6, 28), datetime.datetime(2020, 11, 20, 5, 56, 40), datetime.datetime(2020, 11, 20, 5, 40, 53), datetime.datetime(2020, 11, 20, 5, 33, 20), datetime.datetime(2020, 11, 20, 5, 31, 23), datetime.datetime(2020, 11, 20, 5, 21, 15), datetime.datetime(2020, 11, 20, 5, 10, 44), datetime.datetime(2020, 11, 20, 5, 7, 35), datetime.datetime(2020, 11, 20, 5, 4, 35), datetime.datetime(2020, 11, 20, 5, 4, 3), datetime.datetime(2020, 11, 20, 4, 55, 56), datetime.datetime(2020, 11, 20, 4, 55, 52), datetime.datetime(2020, 11, 20, 4, 55, 47), datetime.datetime(2020, 11, 20, 4, 45,

In [9]:
print(name)

['BORNEONEWS', 'shabani kajala', 'Geisz Chalifah', 'Tribunnews', 'Pikiran Rakyat', 'Moeslim Choice', 'Tan Sri Professor Dato Dr. Asmaliah', 'VOA Indonesia', 'Pikiran Rakyat', 'T.', 'Ms Erca', 'Mulastar', 'Farhad kermanshahi', 'Farhad kermanshahi', 'Mulastar', "Erin's Note", 'inilahcom', 'Panen Saham', 'REPUBLIK MERDEKA | RMOL.ID', 'Rakyat Sulsel Online', 'Rakyat Sulsel Online', 'TJMaxx 🇮🇳🇺🇸', '101.1 MGTRADIO Bandung', 'shiela', 'darkmatter', 'freemalik6', 'Akuratco', 'Asumsi', '20DETIK', 'CNN Indonesia', 'BBC News Swahili', 'detikcom', 'REPUBLIK MERDEKA | RMOL.ID', '🇲🇾Astro AWANI🇲🇾', 'Bisnis.com', '𝓩ulkifli𝓝azri', 'VOA Swahili', 'INDOZONE', 'amir fiqi', 'Kompas.com', 'Berita Terkini', 'Bisnis.com', 'ET News Indonesia', '🇲🇾Astro AWANI🇲🇾', 'Coach Andre Ridho', 'kumparan', 'detikcom', 'Clemente Mimun', 'INDOZONE', 'galamedianews.com', 'Tuit Pekanbaru', 'JPNN.com', 'Kamichi', 'SINDOnews', 'Antara Bengkulu', 'Kompas.com', 'GELORA NEWS', 'Intraday', 'Republika.co.id', 'Antara Jabar', 'Free M

In [10]:
print(screenname)

['BORNEONEWS', 'KajalaShabani', 'GeiszChalifah', 'tribunnews', 'pikiran_rakyat', 'moeslim_choice', '20052000_tan', 'voaindonesia', 'pikiran_rakyat', 'mjstiz', 'ErcaMs', 'Mulastar255', 'Taghbostan2', 'Taghbostan2', 'Mulastar255', 'erinsnote', 'inilahdotcom', 'PanenSaham', 'RMOLOfficial', 'RakyatSulsel', 'RakyatSulsel', 'TJMaxx_72', 'MGT_RADIO', 'shiestheman', 'reconWaRRioRs', 'freemalik611', 'akuratco', 'asumsico', '20detik', 'CNNIndonesia', 'bbcswahili', 'detikcom', 'RMOLOfficial', '501Awani', 'Bisniscom', 'zlk69', 'VOASwahili', 'indozonemedia', 'amirfiqi', 'kompascom', 'BeritaTerkinii', 'Bisniscom', 'ETNewsIndonesia', '501Awani', 'Motifighter', 'kumparan', 'detikcom', 'cjmimun', 'indozonemedia', 'galamedianews', 'TuitPekanbaru', 'jpnncom', 'Justkamichi', 'SINDOnews', 'ANTARABengkulu', 'kompascom', 'geloraco', 'IntradayDotMy', 'republikaonline', 'Antara_Jabar', 'fmtoday', 'antaranews', 'JamesConran1', '20detik', 'RRIPrograma3', 'kompascom', 'kateymccoo', 'HardBoiledEddie', 'devs_new', 

In [11]:
print(location)

['Central Borneo, Indonesia', 'Mara, Tanzania', '', 'Indonesia', 'Bandung, Jawa Barat', 'Jakarta Capital Region, Indone', 'Mekah', 'Washington, DC', 'Bandung, Jawa Barat', 'unknown', '', 'Dar es Salaam, Tanzania', 'zire aseman kabud', 'zire aseman kabud', 'Dar es Salaam, Tanzania', 'DKI Jakarta', 'Jakarta, Indonesia', 'Kelapa Gading-North Jakarta', 'Jakarta Pusat, DKI Jakarta', 'Indonesia', 'Indonesia', '', 'Bandung, West Java, Indonesia', 'Makati City, National Capital Region', '', '', 'DKI Jakarta, Indonesia', 'Jakarta Capital Region', 'JKT-INA', '', 'London', 'Jakarta, Indonesia', 'Jakarta Pusat, DKI Jakarta', 'Malaysia', 'Jakarta, Indonesia', '𝓢ubang𝓙aya @ 𝓾𝓼𝓳', 'Washington, DC', 'Indonesia', 'jakarta', 'Indonesia', 'Indonesia', 'Jakarta, Indonesia', '', 'Malaysia', '#Jakarta #Indonesia', '', 'Jakarta, Indonesia', 'roma ', 'Indonesia', 'Bandung, Indonesia', 'Sosial Media Pekanbaru', 'Indonesia', 'Knoxville, TN', 'Jakarta Pusat, DKI Jakarta', '', 'Indonesia', 'Indonesia', 'Malaysia'

In [12]:
print(acc_created)

[datetime.datetime(2010, 10, 26, 4, 48, 39), datetime.datetime(2017, 9, 23, 18, 55, 42), datetime.datetime(2020, 8, 23, 12, 27, 25), datetime.datetime(2010, 3, 18, 13, 56, 22), datetime.datetime(2009, 7, 21, 18, 48, 20), datetime.datetime(2017, 12, 8, 10, 47, 6), datetime.datetime(2014, 4, 21, 3, 30, 8), datetime.datetime(2009, 9, 17, 18, 19, 35), datetime.datetime(2009, 7, 21, 18, 48, 20), datetime.datetime(2009, 5, 31, 4, 16, 25), datetime.datetime(2020, 9, 1, 5, 1, 54), datetime.datetime(2019, 9, 23, 13, 41, 33), datetime.datetime(2016, 5, 15, 7, 21, 16), datetime.datetime(2016, 5, 15, 7, 21, 16), datetime.datetime(2019, 9, 23, 13, 41, 33), datetime.datetime(2011, 11, 28, 10, 56, 52), datetime.datetime(2009, 6, 12, 16, 51), datetime.datetime(2014, 7, 10, 5, 14, 59), datetime.datetime(2014, 4, 6, 13, 8, 50), datetime.datetime(2012, 5, 12, 13, 34, 9), datetime.datetime(2012, 5, 12, 13, 34, 9), datetime.datetime(2019, 1, 29, 5, 18, 13), datetime.datetime(2009, 7, 21, 19, 9, 36), dateti

In [13]:
print(follower)

[7730, 3223, 38288, 1061769, 427032, 139, 35, 311666, 427032, 37, 189, 51, 554, 554, 51, 1827, 366528, 929, 134928, 17685, 17685, 268, 41532, 191, 1038, 2, 8004, 200951, 82238, 1338019, 702799, 16341299, 134928, 2105161, 1042532, 2121, 19438, 1784426, 228, 7890446, 367, 1042532, 362, 2105161, 1411, 581834, 16341299, 60030, 1784426, 15137, 404, 53899, 2468, 289518, 2394, 7890446, 180290, 2138, 1407706, 3842, 161949, 1138578, 456, 82238, 9022, 7890446, 8, 34, 5, 22598]


In [14]:
print(friend)

[2570, 4995, 128, 519, 18, 276, 178, 391, 18, 573, 262, 128, 856, 856, 128, 834, 4, 9, 195, 29, 29, 398, 384, 703, 29, 35, 17, 3, 10, 23, 171, 30, 195, 333, 36, 2692, 203, 0, 232, 25, 1, 36, 29, 333, 716, 2, 30, 480, 0, 290, 32, 0, 10, 195, 728, 25, 2411, 1, 57, 3109, 3, 141, 894, 10, 102, 25, 29, 724, 41, 2299]


In [15]:
print(verified)

[False, False, False, False, True, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, False, True, True, False, True, False, False, True, False, True, False, True, False, True, True, False, False, False, False, True, False, True, False, True, False, False, True, False, False, True, False, False, True, True, False, False, False, False]


In [21]:
clean_text = data.clean_data(text)
print(clean_text)

['biden tolak trump milu bertanggungjawab', 'kwa hiyo na marekani hawapewi misaada hiyo mhe donald trump analalamika kaibiwa kura anadai kwamba mhe joe biden alikuwa na mabegi ya kura zilizokwisha pigwa zikawa zinatumbukizwa kwenye masanduku kama dunia ingefata akil zenu hakuna nchi hata moja ingepata misaada yenu hiyo', 'ga trump mr joe biden', 'hasil hitung ulang suara georgia trump kalah biden sebut trump presiden tanggung sejarah amerika', 'joe biden donald trump presiden tanggung sejarah as', 'geram biden trump internasional', 'trump eai hiau kuan bi joe biden xi chi kor kai nang bei hiau kuan bi', 'biden strategi trump tolak aku kalah', 'joe biden donald trump presiden tanggung sejarah as', 'sesi masalah ujung perkara trump biden', 'tz nenda kwenye acc ya trump ukitoka hao ingia ya rais mteule biden halafu urudi hapa vidole vipo vitano lakini kamwe havilingani na hauwezi kula muwa kukutana na fundo usiishi maisha ya visasi na ya kutaka kufurahishwa na kl mtu mwanadamu sio mkamili

# Tabel Lastscraping

In [24]:
data.save_sql(clean_text,'lastscraping')

# Tabel Tweet

In [25]:
data.save_sql(clean_text,'tweet')

# Tabel User

In [26]:
data.save_sql(clean_text,'user')

# Reset Status Scraping

In [27]:
data.reset_scraping()

In [28]:
connect.close()